In [2]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from typing import List, Tuple, Dict, Union, Any, Optional
from tqdm import tqdm
try:
    os.chdir(os.path.join(os.getcwd(), '../../Language-Technology-Project'))
    print(os.getcwd())
except:
    print("ALready in current dir")

c:\Users\mo\Documents\Organisations\RUG\Language-Technology-Project


# Generative pipeline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"{device} is available")

cuda is available


In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)

# from transformers import AutoTokenizer, AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained('mosaicml/mpt-1b-redpajama-200b', trust_remote_code=True).to(device)
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

print(f"size_gb(model) = {model.num_parameters() * 4 / 1024**3:.2f} GB")
print("vocab size:", tokenizer.vocab_size)

size_gb(model) = 5.80 GB
vocab size: 50257


In [35]:
def predict_next_token_logits(text):
    encoded_input = tokenizer(text, return_tensors='pt').to(device)
    encoded_input['attention_mask'] = encoded_input['attention_mask'].bool()
    output = model(**encoded_input)
    next_token_logits = output.logits[:, -1, :]
    return next_token_logits

def decode_token_logits(logits):
    return tokenizer.decode(torch.argmax(logits, dim=-1))

text = "Daniel Skala is"
logits = predict_next_token_logits(text)
print(decode_token_logits(logits))


 a


In [38]:
text = "Daniel skala is"
for i in range(16):
    logits = predict_next_token_logits(text)
    text += decode_token_logits(logits)
print(text)

Daniel skala is a member of the Swedish national team and has played for the Swedish national team since


In [500]:
import torch

def get_likelihoods_of_words_given_context(prompt: str, words: list, model, tokenizer):
    """
    Returns a list of likelihoods of words given a prompt.
    """
    logits = predict_next_token_logits(prompt)
    likelihoods = torch.softmax(logits, dim=-1)
    
    word_probs = []
    for word in words:
        tokens = tokenizer.encode(word)
        probability = 1
        for token in tokens:
            probability *= likelihoods[0, token].item()
        # normalize probability so it doesnt penalize longer words
        probability = probability ** (1/len(tokens))
        word_probs.append((word, probability))
    output = sorted(word_probs, key=lambda x: x[1], reverse=True)
    output.sort(key=lambda x: x[1], reverse=True)
    return output
# example
prompt = "2+2="
options = ["2", "3", "4", "5", "6", "7", "8", "9", "10"]
print(get_likelihoods_of_words_given_context(prompt, options, model, tokenizer))
prompt = "Daniel skala is "
options = ["a", "the", "an", "the", "a", "the", "an", "the", "a"]
print(get_likelihoods_of_words_given_context(prompt, options, model, tokenizer))


[('4', 0.20943237841129303), ('5', 0.17149624228477478), ('3', 0.14209160208702087), ('6', 0.06978660076856613), ('7', 0.05673491582274437), ('8', 0.031708039343357086), ('2', 0.027894824743270874), ('9', 0.022482799366116524), ('10', 0.007341871503740549)]
[('a', 8.859507033776026e-06), ('a', 8.859507033776026e-06), ('a', 8.859507033776026e-06), ('an', 8.309674740303308e-06), ('an', 8.309674740303308e-06), ('the', 2.2303545392787782e-06), ('the', 2.2303545392787782e-06), ('the', 2.2303545392787782e-06), ('the', 2.2303545392787782e-06)]


## Category classifier

In [816]:
from settings import categories

class LikelihoodBasedCategoryClassifier:

    def __init__(self, categories):
        self.categories = categories

    def __shuffle_categories(self):
        np.random.shuffle(self.categories)
    
    def __construct_classification_prompt(self, sentence, max_length=32, categories=None):
        prompt = "Categories of Human Value Detection categories: \n\n"
        expectation2category = {}
        for i, category in enumerate(categories):
            name = category["name"]
            description = category["description"]
            description = description[:max_length] + "..." if len(description) > max_length else description
            examples = category["examples"]
            example = examples[np.random.randint(len(examples))]
            example = example[:max_length] + "..." if len(example) > max_length else example
            prompt += f"{name}: {description}\n"
            expectation2category[name] = name
        sentence = sentence[:max_length] + "..." if len(sentence) > max_length else sentence
        prompt += f"\nthe argument \"{sentence}\" is an example of " 

        return prompt, expectation2category

    def classify_once(self, sentence: str) -> List[float]:
        self.__shuffle_categories()
        prompt, expectations = self.__construct_classification_prompt(sentence, categories=self.categories)
        likelihoods = get_likelihoods_of_words_given_context(prompt, list(expectations.keys()), model, tokenizer)
        likelihoods = [(expectations[c], p) for c, p in likelihoods]
        return likelihoods
    
    def __call__(self, sentence: str, trial_count=3) -> List[float]:
        results = {category["name"]: 0.0 for category in self.categories}
        for _ in range(trial_count):
            likelihoods = self.classify_once(sentence)
            for category, likelihood in likelihoods:
                results[category] += likelihood / trial_count
        results = list(results.items())
        results.sort(key=lambda x: x[1], reverse=True)
        return results
    

In [832]:
classifier = LikelihoodBasedCategoryClassifier(categories)
# take a random category 
category = categories[np.random.randint(len(categories))]
sentence = np.random.choice(category["examples"])
print(f"Category: {category['name']}")
print(f"Sentence: {sentence}")

likelihoods = classifier(sentence)
print(likelihoods)
if likelihoods[0][0] == category["name"]:
    print("Correct!")
else:
    print("Incorrect!")

Category: universalism
Sentence: It is good to accept and try to understand those who are different from oneself.
[('hedonism', 0.0010899946531719813), ('achievement', 0.0001290030338562065), ('universalism', 6.0322073592548264e-05), ('self-direction', 5.672701605235671e-05), ('conformity', 3.4866193692424636e-05), ('benevolence', 2.3707310016512084e-05), ('power', 1.6965400694364995e-05), ('tradition', 8.93976837600226e-06), ('stimulation', 6.752418427629308e-06), ('security', 4.980070229976263e-06)]
Incorrect!


In [675]:
# basic evaluation go throguuh all categories and all examples
def evaluate_classifier(classifier, categories, trial_count=5):
    results = {category["name"]: [] for category in categories}
    
    for category in categories:
        for sentence in category["examples"]:
            likelihoods = classifier(sentence, trial_count=trial_count)
            results[category["name"]].append(likelihoods)
    # compute accuracy
    correct = 0
    total = 0
    for category in categories:
        for likelihoods in results[category["name"]]:
            if likelihoods[0][0] == category["name"]:
                correct += 1
            total += 1
    accuracy = correct / total
    return accuracy, results

accuracy, results = evaluate_classifier(classifier, categories)
print(f"Accuracy: {accuracy:.2f}")

Categories of Human Value Detection categories: 

the argument "It is important to uphold traditions." is an example of 
Categories of Human Value Detection categories: 

the argument "It is important to uphold traditions." is an example of 
Categories of Human Value Detection categories: 

the argument "It is important to uphold traditions." is an example of 
Categories of Human Value Detection categories: 

the argument "It is important to uphold traditions." is an example of 
Categories of Human Value Detection categories: 

the argument "It is important to uphold traditions." is an example of 
Categories of Human Value Detection categories: 

the argument "It is important to follow the customs of my religion." is an example of 
Categories of Human Value Detection categories: 

the argument "It is important to follow the customs of my religion." is an example of 
Categories of Human Value Detection categories: 

the argument "It is important to follow the customs of my religion." is